# Farina in Natura - estratégia

## Importando os dados e observações iniciais

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime



In [2]:
from fpdf import FPDF

In [3]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [109]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,total_item_verif,confirm,hora,minuto,time_of_day,day_of_week,day_type
0,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,14.5,False,8,13,manhã,terça-feira,DDS
1,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,15.5,False,8,13,manhã,terça-feira,DDS
2,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,14.5,False,8,14,manhã,terça-feira,DDS
3,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,15.5,False,8,14,manhã,terça-feira,DDS
4,465,2022-02-01 09:01:00,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,39.0,False,9,1,manhã,terça-feira,DDS


In [112]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   venda             25992 non-null  int64         
 1   data              25992 non-null  datetime64[ns]
 2   cliente           25992 non-null  object        
 3   mesa              25992 non-null  object        
 4   produto           25992 non-null  object        
 5   quantidade        25992 non-null  int64         
 6   valor_unit        25992 non-null  float64       
 7   desconto          25992 non-null  float64       
 8   total_item        25992 non-null  float64       
 9   total_item_verif  25992 non-null  float64       
 10  confirm           25992 non-null  bool          
 11  hora              25992 non-null  int64         
 12  minuto            25992 non-null  int64         
 13  time_of_day       25992 non-null  object        
 14  dia_da_semana     2599

# Limpeza e manipulação dos dados

In [80]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda               0
data_hora           0
cliente             0
mesa                0
produto             0
quantidade          0
valor_unit          0
desconto            0
total_item          0
total_item_verif    0
confirm             0
dtype: int64

In [8]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [84]:
# Delete the column "item"

del transactions["item"]

In [ ]:
transactions.info()
transactions.head()

In [11]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [ ]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

In [14]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [21]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [23]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [24]:
# change the decimal delimiter of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [25]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [62]:
# Convert "data_hora" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [89]:
# Creating a horario column that is a copy of data_hora;

# https://www.dataquest.io/blog/python-datetime-tutorial/


transactions['horario'] = (transactions['data_hora'])

#  add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.horario.dt.hour
transactions['minuto'] = transactions.horario.dt.minute

# drop horario column
transactions = transactions.drop(['horario'], axis=1)


### Reamaning the column "data_hora" to "data"

In [96]:
# https://www.statology.org/pandas-rename-columns/
# renaming the column

transactions.rename(columns = {'data_hora':'data'}, inplace = True)


In [92]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'

In [93]:
# Apply time of day function to 'hora' column

transactions['time_of_day'] = transactions.hora.apply(time_of_day)


In [107]:
# https://pt.stackoverflow.com/questions/508484/mostrar-dias-da-semana#:~:text=Como%20weekday()%20retorna%20um,fim%20de%20semana%20ou%20n%C3%A3o.

import locale
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [108]:
# Add a day of week variable and a second classifier of weekday or weekend

from datetime import date
import calendar

transactions['day_of_week'] = [calendar.day_name[i.weekday()] for i in transactions['data']]

transactions['day_type'] = ['FDS' if (i == 'Sunday') | (i == 'Saturday') else 'DDS' for i in 
                                                                            transactions['day_of_week']]


In [111]:
transactions.rename(columns = {'day_of_week':'dia_da_semana'}, inplace = True)

In [113]:
transactions.rename(columns = {'time_of_day':'hora_do_dia'}, inplace = True)

In [114]:
transactions.rename(columns = {'day_type':'tipo_de_dia'}, inplace = True)

In [110]:
# Add an item count variable equal to one for future groupby operations

transactions['item_count'] = 1


In [115]:
transactions.head()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   venda             25992 non-null  int64         
 1   data              25992 non-null  datetime64[ns]
 2   cliente           25992 non-null  object        
 3   mesa              25992 non-null  object        
 4   produto           25992 non-null  object        
 5   quantidade        25992 non-null  int64         
 6   valor_unit        25992 non-null  float64       
 7   desconto          25992 non-null  float64       
 8   total_item        25992 non-null  float64       
 9   total_item_verif  25992 non-null  float64       
 10  confirm           25992 non-null  bool          
 11  hora              25992 non-null  int64         
 12  minuto            25992 non-null  int64         
 13  hora_do_dia       25992 non-null  object        
 14  dia_da_semana     2599

# Verificando alguma divergência dos dados

In [26]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [27]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


In [28]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True].sum()
print (resumo)

venda                                                        13730391
cliente             CONSUMIDORCONSUMIDORCONSUMIDORCONSUMIDORIFOODC...
mesa                M 5M 13M 11M 5NDM 19NDM 13M 13M 17NDM 23M 23M ...
produto             CROISSANTCROISSANTCROISSANTCROISSANTCROISSANT ...
quantidade                                                       2139
valor_unit                                                    21083.7
desconto                                                          0.0
total_item                                                    53878.3
total_item_verif                                              22219.1
confirm                                                          2032
dtype: object


C:\Users\ctobr\AppData\Local\Temp\ipykernel_27660\2200306820.py:2: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  resumo = transactions.loc[transactions["confirm"] == True].sum()


In [30]:
# Gerando um relatório em pdf dos dados divergentes
# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [33]:
# Exporting to a pdf file the transactions that has a inconsistent data.
from matplotlib.backends.backend_pdf import PdfPages


In [34]:
# https://levelup.gitconnected.com/how-to-write-a-pandas-dataframe-as-a-pdf-5cdf7d525488

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis("tight")
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig


In [32]:
# https://www.alixaprodev.com/2022/05/convert-pandas-dataframe-to-pdf-in-python.html
#fig, ax =plt.subplots(figsize=(14,6))
#ax.axis('equal')
#ax.axis('off')
#the_table = ax.table(cellText=ts2.values,colLabels=ts2.columns,loc='center')




In [35]:
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [36]:
### Trying to Align the page number
# https://www.geeksforgeeks.org/matplotlib-figure-figure-text-in-python/
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            #if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                #fig.text(0.5, 0.5/pagesize[0],
                       # "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                       # verticalalignment ='bottom', 
                       # horizontalalignment ='right',
                       #transform = ax.transAxes,
                        #color ='green',
                        #fontsize = 5
                         #)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [38]:
dataframe_to_pdf(ts2, "diverge_01.pdf", numpages=(12, 1), pagesize=(11.7, 8.3))

In [39]:
## Create a HTML file of the resumé "ts2" dataframe.

f = open('exp.html','w')
a = ts2.to_html()
f.write(a)
f.close()

In [116]:
# drop the 'confirm" and "total_item_verif" column
transactions = transactions.drop(['confirm'], axis=1)
transactions = transactions.drop(['total_item_verif'], axis=1)


In [117]:
transactions.head()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   venda          25992 non-null  int64         
 1   data           25992 non-null  datetime64[ns]
 2   cliente        25992 non-null  object        
 3   mesa           25992 non-null  object        
 4   produto        25992 non-null  object        
 5   quantidade     25992 non-null  int64         
 6   valor_unit     25992 non-null  float64       
 7   desconto       25992 non-null  float64       
 8   total_item     25992 non-null  float64       
 9   hora           25992 non-null  int64         
 10  minuto         25992 non-null  int64         
 11  hora_do_dia    25992 non-null  object        
 12  dia_da_semana  25992 non-null  object        
 13  tipo_de_dia    25992 non-null  object        
 14  item_count     25992 non-null  int64         
dtypes: datetime64[ns](1

> # **Exploração dos Dados**

![](Logo_farina_in_natura_edited.jpg)

In [ ]:
# Find Unique Values of each item sold in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
transactions.produto.value_counts()

In [120]:
# find the percentage each item is selling
transactions.produto.value_counts(True)

CROISSANT                         0.092067
PASTEL DE NATA 60G                0.051516
CROISSANT  AMENDOAS 100G          0.028547
FOLHADO MACA 80G                  0.027662
SANDUICHE DE PEPERONI BRIE        0.022930
COOKIES 80G                       0.021507
CROISSANT IFOOD                   0.020545
SOURDOUGH AZEITONA E ALECRIM      0.019891
FOLHADO FRANGO 160G               0.019237
BOMBOLONI DOCE DE LEITE 80G       0.018737
FOCACCIA FORMAGGIO 246G           0.017698
AGUA MINERAL S/GAS 500ML          0.017159
TORRADA FARINA 165G               0.016736
CAFE EQUILIBRADO CAPSULA P        0.016582
SOURDOUGH 369G                    0.016582
LARANJA 400 ML                    0.016544
MINI CROISSANT 25G KIT C 5        0.016428
CROISSANT BRIE PARMA MEL DE       0.016274
PIZZA AL TAGLIO GORGONZOLA E      0.016197
MINI PAIN AU CHOCOLAT 35G         0.016120
BOMBOLONI NUTELLA 80G             0.015736
CROISSANT  TRADICIONAL 80 G       0.015235
QUICHE DE FRANGO ALHO PORO        0.013850
AGUA MINERA

In [60]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
num_itens_vend = transactions.produto.unique().size

print('O número de diferentes itens vendido pela loja é:', num_itens_vend)

O número de diferentes itens vendido pela loja é: 212


In [61]:
# Look at the average number of items purchased...

avg_items = transactions.groupby(by='venda')['produto'].count().mean()

print('A média de itens por venda é: %.2f itens' %avg_items)

A média de itens por venda é: 2.79 itens


In [121]:
# ...and how that varies by time of day

single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['hora','minuto','hora_do_dia','data', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens vendidos por hora do dia é: \n{0}'.format(
    single_transaction.groupby(by='hora_do_dia')['item_count'].mean()))

O número médio de itens vendidos por hora do dia é: 
hora_do_dia
manhã    3.046419
noite    2.663510
tarde    2.783376
Name: item_count, dtype: float64
